In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the path to the directory
directory_path = r'E:\invasive-aquatic-species-data\Invasive_Life_Stage'

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for folder in os.listdir(directory_path):
    #print(class_folder)
    clsp = os.path.join(directory_path,folder)  
    for class_folder in os.listdir(clsp):
        #print(class_folder)
        class_path = os.path.join(clsp, class_folder)    
        #print(class_path)
        for image_file in os.listdir(class_path):
            if image_file.endswith('.png'):
                image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
                img = Image.open(image_path).convert('RGB')
                img = img.resize((32, 32))
                img_array = np.array(img)
            
            # Append image and label to lists
                X_train.append(img_array)
                Y_train.append(class_label)
                #print(class_label)
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


X_train shape: (49227, 32, 32, 3)
Y_train shape: (49227,)


In [2]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
Y_train_encoded = to_categorical(Y_train)
indices = np.arange(len(X_train))
np.random.shuffle(indices)
X_train_shuffled = X_train[indices]
Y_train_encoded_shuffled = Y_train_encoded[indices]
X_train, X_test, y_train, y_test = train_test_split(X_train_shuffled, Y_train_encoded_shuffled, test_size=0.2, random_state=42)

In [3]:
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D

model_path = r"C:\Users\shaif\Downloads\invssl.h5"
model = load_model(model_path)
#model.layers[0].trainable = False
x = model.layers[-8].output  
output = Dense(5, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output)
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=40, mode='max', verbose=1)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = new_model.fit(X_train, y_train, batch_size=32, epochs=50,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
985/985 [==============================] - 77s 60ms/step - loss: 30.4342 - accuracy: 0.4101 - val_loss: 4.9776 - val_accuracy: 0.5510
Epoch 2/50
985/985 [==============================] - 52s 53ms/step - loss: 3.8331 - accuracy: 0.4736 - val_loss: 2.5572 - val_accuracy: 0.5865
Epoch 3/50
985/985 [==============================] - 55s 56ms/step - loss: 2.0222 - accuracy: 0.5281 - val_loss: 1.0872 - val_accuracy: 0.5800
Epoch 4/50
985/985 [==============================] - 56s 57ms/step - loss: 1.5799 - accuracy: 0.5510 - val_loss: 1.1352 - val_accuracy: 0.5911
Epoch 5/50
985/985 [==============================] - 59s 60ms/step - loss: 1.1786 - accuracy: 0.5808 - val_loss: 1.0781 - val_accuracy: 0.5522
Epoch 6/50
985/985 [==============================] - 56s 57ms/step - loss: 1.2305 - accuracy: 0.5895 - val_loss: 1.0635 - val_accuracy: 0.6048
Epoch 7/50
985/985 [==============================] - 61s 62ms/step - loss: 1.2049 - accuracy: 0.6165 - val_loss: 1.0110 - val_accuracy

In [ ]:
history = new_model.fit(X_train, y_train, batch_size=32, epochs=2,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
985/985 [==============================] - 59s 60ms/step - loss: 0.9954 - accuracy: 0.7572 - val_loss: 610.0840 - val_accuracy: 0.7565
Epoch 2/50
985/985 [==============================] - 64s 65ms/step - loss: 0.7025 - accuracy: 0.7658 - val_loss: 73.8275 - val_accuracy: 0.7629
Epoch 3/50
985/985 [==============================] - 62s 63ms/step - loss: 0.7918 - accuracy: 0.7715 - val_loss: 14644.0547 - val_accuracy: 0.5303
Epoch 4/50
985/985 [==============================] - 60s 61ms/step - loss: 0.9408 - accuracy: 0.7575 - val_loss: 476.9403 - val_accuracy: 0.7543
Epoch 5/50
985/985 [==============================] - 55s 56ms/step - loss: 0.6378 - accuracy: 0.7722 - val_loss: 444.2742 - val_accuracy: 0.7479
Epoch 6/50
985/985 [==============================] - 58s 59ms/step - loss: 0.6438 - accuracy: 0.7767 - val_loss: 288.0295 - val_accuracy: 0.7673
Epoch 7/50
985/985 [==============================] - 53s 54ms/step - loss: 1.5090 - accuracy: 0.7093 - val_loss: 2223.5852

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

Test Balanced Accuracy: 0.567010311131761
Test Accuracy: 0.6581352833638026


In [5]:
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x)
ne_model = Model(inputs=base_model.input, outputs=output)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, mode='max', verbose=1)
ne_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = ne_model.fit(X_train, y_train, batch_size=32, epochs=50,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
985/985 [==============================] - 66s 61ms/step - loss: 1.3989 - accuracy: 0.6972 - val_loss: 1.3019 - val_accuracy: 0.7028
Epoch 2/50
985/985 [==============================] - 50s 50ms/step - loss: 0.7277 - accuracy: 0.7528 - val_loss: 0.8182 - val_accuracy: 0.7375
Epoch 3/50
985/985 [==============================] - 53s 54ms/step - loss: 0.6014 - accuracy: 0.7977 - val_loss: 1.3534 - val_accuracy: 0.4752
Epoch 4/50
985/985 [==============================] - 55s 56ms/step - loss: 0.5050 - accuracy: 0.8255 - val_loss: 0.5305 - val_accuracy: 0.7947
Epoch 5/50
985/985 [==============================] - 68s 69ms/step - loss: 0.6689 - accuracy: 0.7966 - val_loss: 3.0955 - val_accuracy: 0.3920
Epoch 6/50
985/985 [==============================] - 84s 85ms/step - loss: 0.5568 - accuracy: 0.8068 - val_loss: 1.9077 - val_accuracy: 0.4870
Epoch 7/50
985/985 [==============================] - 93s 95ms/step - loss: 0.5315 - accuracy: 0.8155 - val_loss: 0.8604 - val_accuracy:

In [6]:
from sklearn.metrics import accuracy_score
predictions = ne_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.864411943936624
Test Balanced Accuracy: 0.863739992105631
Test Accuracy: 0.864411943936624


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNetV2(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x) 
new_model = Model(inputs=base_model.input, outputs=output)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

In [ ]:
from sklearn.metrics import accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)